In [1]:
# Step 1: Import libraries, read dataset

import pandas as pd
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
spacex_df = pd.read_csv('/Users/bradczh/Documents/20 Career/IBM Data Science/Course 10 - Applied Data Science Capstone/Module 3 - Interactive Visual Analytics and Dashboard/spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [2]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label':'All Sites', 'value':'ALL'},
                                                     {'label':'CCAFS LC-40', 'value':'CCAFLC'},
                                                     {'label':'VAFBSLC-4E', 'value':'VAFB'},
                                                     {'label':'KSC LC-39A', 'value':'FSC'},
                                                     {'label':'CCAFS SLC-40', 'value':'CCAFSLC'}],
                                            value='ALL',
                                            placeholder='Select a Launch Site here',
                                            searchable=True,
                                            style={'width':'100%', 'padding':3, 'font-size':20, 'text-align':'left'}),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, 
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [3]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
   # Define a helper function to create the pie chart 
    def create_pie_chart(data, title, value_col='class', name_col='Launch Site'): 
        fig = px.pie(data, values=value_col, names=name_col, title=title) 
        return fig
        
    site_mapping = {'CCAFLC':'CCAFS LC-40', 
                    'VAFB':'VAFB SLC-4E', 
                    'FSC':'KSC LC-39A', 
                    'CCAFSLC':'CCAFS SLC-40'}
    
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['class']==1]
        data = filtered_df.groupby(['Launch Site']).agg({'class':'count'}).reset_index()
        return create_pie_chart(data, 'Total Success Launches for Site')
    elif entered_site in site_mapping: 
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_mapping[entered_site]] 
        data = filtered_df.groupby(['class']).agg({'Launch Site': 'count'}).reset_index() 
        title = f'Total Success Launches by Site: {site_mapping[entered_site]}' 
        return create_pie_chart(data, title, value_col='Launch Site', name_col='class')

In [4]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, slider_range):
    
    # Define a helper function to create the scatter plot 
    def create_scatter_plot(data, title): 
        low, high = slider_range 
        mask = (data['Payload Mass (kg)'] >= low) & (data['Payload Mass (kg)'] <= high) 
        fig = px.scatter(data[mask], x='Payload Mass (kg)', y='class', 
                         color='Booster Version Category', title=title) 
        return fig
    
    site_mapping = {'CCAFLC': 'CCAFS LC-40', 
                    'VAFB': 'VAFBSLC-4E', 
                    'FSC': 'KSC LC-39A', 
                    'CCAFSLC': 'CCAFS SLC-40' }
    
    if entered_site == 'ALL' and slider_range:
        filtered_df = spacex_df
        return create_scatter_plot(filtered_df, 'Correlation between Payload and Success for All Sites')
    elif entered_site in site_mapping and slider_range: 
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_mapping[entered_site]] 
        title = f'Correlation between Payload and Success for Site: {site_mapping[entered_site]}' 
        return create_scatter_plot(filtered_df, title)

In [5]:
# Run the app
if __name__ == '__main__':
    app.run_server()